In [1]:
# Import the Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [2]:
# Create a new set of 2,000 random latitudes and longitudes. 
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)


In [3]:
# Add the latitudes and longitudes to a list
coordinates = list (lat_lngs)

In [6]:
# Create list for cities
cities = []

# Identify nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If City is unique, then add to cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities generated: {len(cities)}")

Number of cities generated: 731


In [4]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_country = city_weather["sys"]["country"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append the city information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | punta arenas
Processing Record 2 of Set 1 | airai
Processing Record 3 of Set 1 | champerico
Processing Record 4 of Set 1 | pinheiro machado
Processing Record 5 of Set 1 | atuona
Processing Record 6 of Set 1 | jamestown
Processing Record 7 of Set 1 | kallifitos
Processing Record 8 of Set 1 | wanning
Processing Record 9 of Set 1 | banjar
Processing Record 10 of Set 1 | mabaruma
Processing Record 11 of Set 1 | nyurba
Processing Record 12 of Set 1 | bluff
Processing Record 13 of Set 1 | mataura
Processing Record 14 of Set 1 | revelstoke
Processing Record 15 of Set 1 | luderitz
Processing Record 16 of Set 1 | yusva
City not found. Skipping...
Processing Record 17 of Set 1 | katsuura
Processing Record 18 of Set 1 | albany
Processing Record 19 of Set 1 | ribeira grande
Processing Record 20 of Set 1 | hermanus
Processing Record 21 of Set 1 | grindavik
Processing Record 22 of Set 1 | ushuaia
Processing Re

Processing Record 43 of Set 4 | ajdabiya
Processing Record 44 of Set 4 | baykit
Processing Record 45 of Set 4 | salalah
Processing Record 46 of Set 4 | broome
Processing Record 47 of Set 4 | adrar
Processing Record 48 of Set 4 | mgandu
Processing Record 49 of Set 4 | havelock
Processing Record 50 of Set 4 | avera
Processing Record 1 of Set 5 | saskylakh
Processing Record 2 of Set 5 | bosaso
Processing Record 3 of Set 5 | los llanos de aridane
Processing Record 4 of Set 5 | el reno
Processing Record 5 of Set 5 | kokstad
Processing Record 6 of Set 5 | kawalu
Processing Record 7 of Set 5 | marzuq
Processing Record 8 of Set 5 | ligayan
Processing Record 9 of Set 5 | grajau
Processing Record 10 of Set 5 | sakaiminato
Processing Record 11 of Set 5 | minbu
Processing Record 12 of Set 5 | tete
Processing Record 13 of Set 5 | shenjiamen
Processing Record 14 of Set 5 | sentyabrskiy
City not found. Skipping...
Processing Record 15 of Set 5 | vagay
Processing Record 16 of Set 5 | naze
Processing R

Processing Record 34 of Set 8 | xiongshi
City not found. Skipping...
Processing Record 35 of Set 8 | muravlenko
Processing Record 36 of Set 8 | xuddur
Processing Record 37 of Set 8 | armidale
Processing Record 38 of Set 8 | okhotsk
Processing Record 39 of Set 8 | kembe
Processing Record 40 of Set 8 | chapais
Processing Record 41 of Set 8 | kudahuvadhoo
Processing Record 42 of Set 8 | west bay
Processing Record 43 of Set 8 | ishigaki
Processing Record 44 of Set 8 | mantua
Processing Record 45 of Set 8 | zemaiciu naumiestis
Processing Record 46 of Set 8 | kazalinsk
City not found. Skipping...
Processing Record 47 of Set 8 | maceio
Processing Record 48 of Set 8 | havoysund
Processing Record 49 of Set 8 | champasak
Processing Record 50 of Set 8 | bolungarvik
City not found. Skipping...
Processing Record 1 of Set 9 | san policarpo
Processing Record 2 of Set 9 | san quintin
Processing Record 3 of Set 9 | plettenberg bay
Processing Record 4 of Set 9 | creazzo
Processing Record 5 of Set 9 | to

Processing Record 21 of Set 12 | ulaangom
Processing Record 22 of Set 12 | louisbourg
City not found. Skipping...
Processing Record 23 of Set 12 | pedasi
Processing Record 24 of Set 12 | menongue
Processing Record 25 of Set 12 | guapo
Processing Record 26 of Set 12 | barawe
City not found. Skipping...
Processing Record 27 of Set 12 | malakal
Processing Record 28 of Set 12 | rangpur
Processing Record 29 of Set 12 | baiao
Processing Record 30 of Set 12 | chirongui
Processing Record 31 of Set 12 | tsaratanana
Processing Record 32 of Set 12 | kirovskiy
Processing Record 33 of Set 12 | solnechnyy
Processing Record 34 of Set 12 | prince rupert
Processing Record 35 of Set 12 | kon tum
Processing Record 36 of Set 12 | abu samrah
Processing Record 37 of Set 12 | soskovo
Processing Record 38 of Set 12 | bowen
Processing Record 39 of Set 12 | kamenskoye
City not found. Skipping...
Processing Record 40 of Set 12 | sekoma
Processing Record 41 of Set 12 | dibrugarh
Processing Record 42 of Set 12 | l

In [9]:
len(city_data)

681

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)



,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Punta Arenas,CL,-53.1500,-70.9167,46.51,87,75,5.75,broken clouds
1,Airai,TL,-8.9266,125.4092,60.26,89,100,1.61,light rain
2,Champerico,GT,14.3000,-91.9167,95.54,62,75,12.97,broken clouds
3,Pinheiro Machado,BR,-31.5783,-53.3811,64.69,60,100,6.04,overcast clouds
4,Atuona,PF,-9.8000,-139.0333,79.57,80,18,18.19,light rain
5,Jamestown,US,42.0970,-79.2353,41.05,39,6,5.99,clear sky
6,Kallifitos,GR,41.1667,24.2167,48.90,91,100,6.33,overcast clouds
7,Wanning,CN,18.8003,110.3967,75.67,89,59,5.86,broken clouds
8,Banjar,ID,-8.1900,114.9675,78.17,90,100,3.09,overcast clouds
9,Mabaruma,GY,8.2000,-59.7833,85.30,56,99,8.57,overcast clouds


In [12]:
# Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")